<a href="https://colab.research.google.com/github/JasmineGhumman/SignLanguageDetection/blob/master/softmax.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
# Install the PyDrive wrapper & import libraries.
# This only needs to be done once per notebook.
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# Authenticate and create the PyDrive client.
# This only needs to be done once per notebook.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)



     |████████████████████████████████| 993kB 51.5MB/s 


In [0]:
# Download a file based on its file ID.
#
# A file ID looks like: laggVyWshwcyP6kEI-y_W3P8D26sz
'''
file_id = '1sNVHpcnPHZiDaSQ6DhhxY1ZbdEByi5gk'
downloaded = drive.CreateFile({'id': file_id})
print('Downloaded content "{}"'.format(downloaded.GetContentString()))
'''

fid = drive.ListFile({'q':"title='mydata.zip'"}).GetList()[0]['id']
f = drive.CreateFile({'id': fid})
f.GetContentFile('mydata.zip')

In [0]:
!unzip mydata

In [0]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os


# Any results you write to the current directory are saved as output.import numpy as np # linear algebra


In [7]:
#!pip uninstall keras
#!pip install keras==2.0.9
from keras.models import Sequential
from keras.layers import Convolution2D
from keras.layers import MaxPooling2D
from keras.layers import Flatten
from keras.layers import Dense, Dropout
from keras import optimizers
from keras.models import Model
from keras.layers import (Activation, Dropout, Flatten, Dense, GlobalMaxPooling2D,GlobalAveragePooling2D,
                          BatchNormalization, Input, Conv2D, Concatenate)
'''from keras.applications import InceptionResNetV2
conv_base= InceptionResNetV2(weights='imagenet',include_top=False,input_shape=(75,75,3))'''

import keras
import numpy as np
from keras.applications.resnet50 import ResNet50

Using TensorFlow backend.


In [8]:
res_weights_path = "resnet50_weights.hd"
print("[INFO] loading .." )
#model = ResNet50(include_top=False,input_shape=(64, 64, 3))
base_model =ResNet50(weights='imagenet', include_top=False)
#my_model_2 = ResNet50(weights='imagenet')
#base_model.load_weights('../input/ResNet-50/resnet50_weights_tf_dim_ordering_tf_kernels_notop.h5')
# include_top= False means that we won’t be keeping the Fully-Connected (FC) layers at the end of the model


x0 = base_model.output
x1 = GlobalAveragePooling2D()(x0)
x2 = GlobalMaxPooling2D()(x0)
x = Concatenate()([x1,x2])
x = BatchNormalization()(x)
x = Dropout(0.7)(x)
x = Dense(2048, activation='relu')(x)
x = BatchNormalization()(x)
x = Dropout(0.7)(x)
predictions = Dense(26, activation='softmax')(x)
model = Model(inputs=base_model.input, outputs=predictions)

[INFO] loading ..
Instructions for updating:
Colocations handled automatically by placer.


/usr/local/lib/python3.6/dist-packages/keras_applications/resnet50.py:265: UserWarning: The output shape of `ResNet50(include_top=False)` has been changed since Keras 2.2.0.
  warnings.warn('The output shape of `ResNet50(include_top=False)` '


94658560/94653016 [==============================] - 8s 0us/step
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [0]:
for layer in model.layers:
    layer.trainable = False

for i in range(-6,0):
    model.layers[i].trainable = True

model.compile(
    loss='binary_crossentropy',
    optimizer=optimizers.SGD(lr = 0.01),metrics=['accuracy'])

#model.summary()

In [10]:
#Part 2 Fittting the CNN to the image
from keras.preprocessing.image import ImageDataGenerator
train_datagen = ImageDataGenerator(
        rescale=1./255,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True)

test_datagen = ImageDataGenerator(rescale=1./255)

training_set = train_datagen.flow_from_directory(
        'mydata/train',
        target_size=(224, 224),
        batch_size=32,
        class_mode='categorical')

test_set = test_datagen.flow_from_directory(
        'mydata/test',
        target_size=(224, 224),
        batch_size=32,
        class_mode='categorical')


Found 68250 images belonging to 26 classes.
Found 9775 images belonging to 26 classes.


In [11]:
moDel = model.fit_generator(
        training_set,
        steps_per_epoch=900,
        epochs=10,
        validation_data = test_set,
        validation_steps = 200
      )

#Saving the model
import h5py
model.save('Trained_model.h5')
modelcheckpoint = keras.callbacks.ModelCheckpoint("keras.model",verbose=1)


Instructions for updating:
Use tf.cast instead.
Epoch 1/10
900/900 [==============================] - 412s 458ms/step - loss: 0.2407 - acc: 0.9483 - val_loss: 0.1827 - val_acc: 0.9615
Epoch 2/10
900/900 [==============================] - 401s 446ms/step - loss: 0.1925 - acc: 0.9520 - val_loss: 0.1830 - val_acc: 0.9615
Epoch 3/10
900/900 [==============================] - 403s 448ms/step - loss: 0.1596 - acc: 0.9563 - val_loss: 0.1856 - val_acc: 0.9615
Epoch 4/10
900/900 [==============================] - 404s 448ms/step - loss: 0.1380 - acc: 0.9598 - val_loss: 0.1876 - val_acc: 0.9615
Epoch 5/10
900/900 [==============================] - 405s 450ms/step - loss: 0.1243 - acc: 0.9626 - val_loss: 0.1888 - val_acc: 0.9615
Epoch 6/10
900/900 [==============================] - 403s 448ms/step - loss: 0.1108 - acc: 0.9651 - val_loss: 0.1899 - val_acc: 0.9615
Epoch 7/10
900/900 [==============================] - 402s 447ms/step - loss: 0.1025 - acc: 0.9672 - val_loss: 0.1928 - val_acc: 0.9615


In [0]:
#download model from lhs files.Save on computer. t